# 절차

- 시퀀스준비 -> add() -> compile() -> fit() -> 평가

In [0]:
import keras
from keras import models, layers, backend
from keras.datasets import mnist
%tensorflow_version 1.x

# 데이터 확보,준비

In [0]:
(x_train, y_train),(x_test,y_test)=mnist.load_data()


In [0]:
x_train.shape, y_train.shape
#y_train, y_test는 벡터화 처리해야 한다

((60000, 28, 28), (60000,))

In [0]:
#이미지 크기
img_rows, img_cols= x_train.shape[1:]

In [0]:
#채널을 설명하는 값에 의해 데이터에 채널값을 앞 혹은 뒤에 추가하여 dimension을 확장<-tensorflow에서는 안보였던 부분
backend.image_data_format()

'channels_last'

In [0]:
#입력데이터보정(reshape)
if backend.image_data_format()=='channels_first':
  #[60000,28,28]->[60000,1,28,28]
  x_train=x_train.reshape(x_train.shape[0],1,img_rows,img_cols)
  x_test=x_test.reshape(x_test.shape[0],1,img_rows,img_cols)
  input_shape=(1,img_rows,img_cols)
else:
  #[60000,28,28]->[60000,28,28,1] <- 텐서플로우의 CNN예제의 x의 shape이었다
  x_train=x_train.reshape(x_train.shape[0],img_rows,img_cols,1)
  x_test=x_test.reshape(x_test.shape[0],img_rows,img_cols,1)
  input_shape=(img_rows,img_cols,1)

In [0]:
#배열의 형변환
x_train=x_train.astype('float32')
x_test=x_test.astype('float32')

In [0]:
#데이터의 정규화, 
x_train/=255
x_test /=255

In [0]:
#레이블 데이터 -> 벡터화
y_train[:10]

array([5, 0, 4, 1, 9, 2, 1, 3, 1, 4], dtype=uint8)

In [0]:
y_train=keras.utils.to_categorical(y_train, 10)
y_test=keras.utils.to_categorical(y_test, 10) 
y_train.shape,y_train[0]

((60000, 10), array([0., 0., 0., 0., 0., 1., 0., 0., 0., 0.], dtype=float32))

# 레이어 설계

- 시퀀스->add->compile->fit->평가

In [0]:
#모델준비(시퀀스)
model=models.Sequential()

In [0]:
#합성곱층 추가:채널수 증가, 특징 추출
# input_shape:입력데이터의 shape
model.add(layers.Conv2D(32,kernel_size=(3,3),strides=(1,1),padding='valid',activation='relu', input_shape=input_shape))

In [0]:
#풀링층 추가:이미지사이즈 감소, 특징강화
model.add(layers.MaxPool2D(pool_size=(2,2),strides=None))

In [0]:
#과적합 방지:데이터에 적응되는것을 방지, 학습을 방해
model.add(layers.Dropout(0.25))


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [0]:
#합성곱층 추가:채널수 증가, 특징 추출
model.add(layers.Conv2D(64,(3,3),activation='relu'))

In [0]:
#풀링층 추가:이미지사이즈 감소, 특징강화
model.add(layers.MaxPool2D(pool_size=(2,2)))

In [0]:
#과적합 방지:데이터에 적응되는것을 방지, 학습을 방해
model.add(layers.Dropout(0.25))

In [0]:
#데이터를 편다 -> flatten 처리
model.add(layers.Flatten())

In [0]:
#전결합층:데이터를 1차로 펴준다
#7x7x64->flatten->128에수렴
model.add(layers.Dense(128,activation='relu'))

In [0]:
#과적합 방지:데이터에 적응되는것을 방지, 학습을 방해
model.add(layers.Dropout(0.5))

In [0]:
#출력층:데이터10개로 수렴
model.add(layers.Dense(10,activation='softmax'))

# 학습 및 평가


In [0]:
model.compile(loss=keras.losses.categorical_crossentropy,optimizer='rmsprop',metrics=['accuracy'])

In [0]:
epochs=10
batch_size=128

In [0]:
model.fit(x_train,y_train,batch_size=batch_size,epochs=epochs,validation_split=0.25)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Train on 45000 samples, validate on 15000 samples
Epoch 1/10





45000/45000 [==============================] - 41s 904us/step - loss: 0.3970 - acc: 0.8744 - val_loss: 0.0941 - val_acc: 0.9707
Epoch 2/10
45000/45000 [==============================] - 40s 885us/step - loss: 0.1287 - acc: 0.9612 - val_loss: 0.0633 - val_acc: 0.9815
Epoch 3/10
45000/45000 [==============================] - 40s 885us/step - loss: 0.0967 - acc: 0.9710 - val_loss: 0.0602 - val_acc: 0.9807
Epoch 4/10
45000/45000 [==============================] - 40s 885us/step - loss: 0.0784 - acc: 0.9760 - val_loss: 0.0478 - val_acc: 0.9850
Epoch 5/10
45000/45000 [==============================] - 40s 890us/step - loss: 0.0687 - acc: 0.9798 - val_loss: 0.0437 - val_acc: 0.9869
Epoch 6/10
45000/45000 [==============================] - 40s 886us/step - loss: 0.0600 - acc: 0.9818 - val_loss: 0.0400 - val_acc: 0.9877
Epoch 7/10
450

In [0]:
#점수
score=model.evaluate(x_test,y_test)
score

10000/10000 [==============================] - 3s 295us/step


[0.026709363029536325, 0.991]